In [ ]:
import os
import pickle


def move_videos():
    EMOTIONS = [
        "angry",
        "calm",
        "disgust",
        "fear",
        "sad",
        "happy",
        "neutral",
        "surprise",
    ]

    for f in os.listdir("Videos/"):
        if f not in EMOTIONS:
            ids = f.split("-")
            if ids[0] == "01":
                os.remove("Videos/" + f)
            else:
                if ids[2] == "01":
                    os.rename("Videos/" + f, "Videos/neutral/" + f)
                elif ids[2] == "02":
                    os.rename("Videos/" + f, "Videos/calm/" + f)
                elif ids[2] == "03":
                    os.rename("Videos/" + f, "Videos/happy/" + f)
                elif ids[2] == "04":
                    os.rename("Videos/" + f, "Videos/sad/" + f)
                elif ids[2] == "05":
                    os.rename("Videos/" + f, "Videos/angry/" + f)
                elif ids[2] == "06":
                    os.rename("Videos/" + f, "Videos/fear/" + f)
                elif ids[2] == "07":
                    os.rename("Videos/" + f, "Videos/disgust/" + f)
                elif ids[2] == "08":
                    os.rename("Videos/" + f, "Videos/surprise/" + f)


def save_data():
    data_dict = {
        "EMOTIONS": [f for f in os.listdir("Videos/") if os.path.isdir("Videos/" + f)],
        "VIDEO_PATH": "Videos/",
        "DATA_PATH": "data/",
        "SEQUENCE_PATH": "sequence/"
    }
    with open("basic_emotions_data.pkl", "wb") as f:
        # python3
        pickle.dump(data_dict, f)
        # python2
        # pickle.dump(data_dict, f, protocol=2)


def load_var():
    if not os.path.exists('basic_emotions_data.pkl'):
        save_data()
    return pickle.load(open('basic_emotions_data.pkl', 'rb'))


move_videos()
save_data()

In [ ]:
import pandas as pd


# Using OpenFace to get frames, au analysis, and facial landmark locations
def get_facial_analysis():
    for emotion in EMOTIONS:
        print(emotion)
        emotion_path = VIDEO_PATH + emotion + '/'
        data_path = DATA_PATH + emotion + '/'
        if not os.path.exists(data_path):
            os.makedirs(data_path)
        for video in os.listdir(emotion_path):
            video_file = emotion_path + video
            save_path = data_path + video.replace('.mp4', '') + '/'
            command = (
                    "./OpenFace/build/bin/FeatureExtraction -f "
                    + video_file
                    + " -out_dir "
                    + save_path
            )
            os.system(command)


# Remove not recognised frames
def clean_frames():
    for emotion in EMOTIONS:
        print(emotion)
        video_list = [f for f in os.listdir(DATA_PATH + emotion)]
        for video in video_list:
            video_path = DATA_PATH + emotion + "/" + video + "/" + video + "_aligned"
            csv_path = DATA_PATH + emotion + "/" + video + "/" + video + ".csv"
            df = pd.read_csv(csv_path)
            df = df[df[" success"] != 1].loc[:, "frame"]
            to_delete = df.tolist()
            to_delete = [int(f) for f in to_delete]
            frames = [
                f
                for f in os.listdir(video_path)
                if os.path.isfile(os.path.join(video_path, f))
            ]
            for frame in frames:
                number = int(frame.split("frame_det_00_")[1].split(".bmp")[0])
                if number in to_delete:
                    print(os.path.join(video_path, frame))
                    os.remove(os.path.join(video_path, frame))



DATA = load_var()
EMOTIONS = list(DATA['EMOTIONS'])
VIDEO_PATH = DATA['VIDEO_PATH']
DATA_PATH = DATA['DATA_PATH']
get_facial_analysis()
clean_frames()